In [1]:
import pandas as pd
import time
from collections import defaultdict

In [2]:
%%time
train = pd.read_csv('./train_preprocess2.csv',
                    dtype={
                        'C1_fix':object,
                        'banner_pos_fix':object,
                        'site_id_fix_fix':object,
                        'site_domain_fix_fix':object,
                        'site_category_fix_fix':object,
                        'app_id_fix_fix':object,
                        'app_domain_fix_fix':object,
                        'app_category_fix_fix':object,
                        'device_id_fix_fix':object,
                        'device_ip_fix_fix':object,
                        'device_model_fix_fix':object,
                        'device_type_fix':object,
                        'device_conn_type_fix':object,
                        'C14_fix_fix':object,
                        'C15_fix':object,
                        'C16_fix':object,
                        'C17_fix_fix':object,
                        'C18_fix':object,
                        'C19_fix_fix':object,
                        'C20_fix_fix':object,
                        'C21_fix':object,
                        'hour_fix_fix':object
                    })
train.shape

CPU times: user 1min 56s, sys: 25.2 s, total: 2min 21s
Wall time: 2min 24s


(40428967, 24)

In [3]:
train.dtypes

id                       float64
click                      int64
C1_fix                    object
banner_pos_fix            object
site_id_fix_fix           object
site_domain_fix_fix       object
site_category_fix_fix     object
app_id_fix_fix            object
app_domain_fix_fix        object
app_category_fix_fix      object
device_id_fix_fix         object
device_ip_fix_fix         object
device_model_fix_fix      object
device_type_fix           object
device_conn_type_fix      object
C14_fix_fix               object
C15_fix                   object
C16_fix                   object
C17_fix_fix               object
C18_fix                   object
C19_fix_fix               object
C20_fix_fix               object
C21_fix                   object
hour_fix_fix              object
dtype: object

In [4]:
f_cols = train.columns.tolist()
f_cols.remove('id')
f_cols.remove('click')

In [5]:
%%time
train[f_cols].nunique()

CPU times: user 1min 36s, sys: 19.7 s, total: 1min 55s
Wall time: 1min 57s


C1_fix                     4
banner_pos_fix             4
site_id_fix_fix           27
site_domain_fix_fix       25
site_category_fix_fix      4
app_id_fix_fix            17
app_domain_fix_fix         8
app_category_fix_fix       3
device_id_fix_fix          2
device_ip_fix_fix         14
device_model_fix_fix      95
device_type_fix            3
device_conn_type_fix       2
C14_fix_fix              116
C15_fix                    2
C16_fix                    3
C17_fix_fix               64
C18_fix                    2
C19_fix_fix               14
C20_fix_fix               22
C21_fix                   17
hour_fix_fix              24
dtype: int64

## 模型訓練

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.preprocessing import StandardScaler

In [7]:
%%time
train_distinct = train.drop(columns=['id']).drop_duplicates()
train_distinct.shape

CPU times: user 1min 17s, sys: 33.2 s, total: 1min 51s
Wall time: 1min 54s


(3234119, 23)

In [22]:
train_distinct.click.value_counts()

0    2170735
1    1063384
Name: click, dtype: int64

In [23]:
%%time
train_sample = train_distinct.sample(frac=0.3, replace=True, random_state=42)
train_sample.shape

CPU times: user 3.31 s, sys: 83.6 ms, total: 3.39 s
Wall time: 3.41 s


(970236, 23)

In [24]:
train_sample.click.value_counts()

0    650447
1    319789
Name: click, dtype: int64

In [11]:
train_sample[f_cols].nunique()

C1_fix                     4
banner_pos_fix             4
site_id_fix_fix           27
site_domain_fix_fix       25
site_category_fix_fix      4
app_id_fix_fix            17
app_domain_fix_fix         8
app_category_fix_fix       3
device_id_fix_fix          2
device_ip_fix_fix         14
device_model_fix_fix      95
device_type_fix            3
device_conn_type_fix       2
C14_fix_fix              116
C15_fix                    2
C16_fix                    3
C17_fix_fix               64
C18_fix                    2
C19_fix_fix               14
C20_fix_fix               22
C21_fix                   17
hour_fix_fix              24
dtype: int64

In [25]:
train_sample.head()

,click,C1_fix,banner_pos_fix,site_id_fix_fix,site_domain_fix_fix,site_category_fix_fix,app_id_fix_fix,app_domain_fix_fix,app_category_fix_fix,device_id_fix_fix,...,device_conn_type_fix,C14_fix_fix,C15_fix,C16_fix,C17_fix_fix,C18_fix,C19_fix_fix,C20_fix_fix,C21_fix,hour_fix_fix
24138531,1,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,0,20213,DNP,DNP,2316,DNP,167,100081,16,3
32776850,1,1005,0,DNP,DNP,DNP,DNP,DNP,DNP,a99f214a,...,0,22257,DNP,DNP,2545,DNP,167,-1,221,3
24318109,0,1005,0,DNP,DNP,DNP,9c13b419,DNP,f95efa07,a99f214a,...,0,18092,DNP,DNP,2060,DNP,39,-1,23,DNP
26586518,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,0,22676,DNP,DNP,2616,DNP,35,100083,51,19
14957287,0,1005,1,DNP,DNP,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,0,DNP,DNP,DNP,DNP,2,39,100233,32,11


In [26]:
x = train_sample.drop(columns=['click'])
y = train_sample[['click']]

In [27]:
%%time
x = pd.get_dummies(x)
x.shape, y.shape

CPU times: user 4.54 s, sys: 465 ms, total: 5 s
Wall time: 5.02 s


((970236, 472), (970236, 1))

In [28]:
%%time
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

CPU times: user 3.69 s, sys: 550 ms, total: 4.24 s
Wall time: 4.27 s


In [ ]:
%%time
para_search = {'criterion':['gini','entropy'],'max_depth':[i for i in range(20, 50, 5)]}
gs = GridSearchCV(dt(), param_grid=para_search, scoring='recall', verbose=1, n_jobs=-1)
gs.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [19]:
gs.best_params_

{'criterion': 'gini', 'max_depth': 18}

In [20]:
gs.best_score_

0.10828391975957437